<a href="https://colab.research.google.com/github/regiiis/dlv_group_project/blob/main/dlv_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DLV Group Project

In [5]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_json("/content/drive/My Drive/Colab Notebooks/data/InfraredSolarModules/module_metadata.json",orient="index")
df

,image_filepath,anomaly_class
13357,images/13357.jpg,No-Anomaly
13356,images/13356.jpg,No-Anomaly
19719,images/19719.jpg,No-Anomaly
11542,images/11542.jpg,No-Anomaly
11543,images/11543.jpg,No-Anomaly
...,...,...
8488,images/8488.jpg,Vegetation
8489,images/8489.jpg,Vegetation
7464,images/7464.jpg,Cracking
18065,images/18065.jpg,No-Anomaly


In [9]:
img1 = mpimg.imread('/content/drive/My Drive/Colab Notebooks/data/InfraredSolarModules/images/13357.jpg')
img2 = mpimg.imread('/content/drive/My Drive/Colab Notebooks/data/InfraredSolarModules/images/8488.jpg')
img3 = mpimg.imread('/content/drive/My Drive/Colab Notebooks/data/InfraredSolarModules/images/7464.jpg')
imgplot = plt.imshow(img1)
plt.show()
imgplot = plt.imshow(img2)
plt.show()
imgplot = plt.imshow(img3)
plt.show()

FileNotFoundError: ignored